In [31]:
import pandas as pd
import requests
from tqdm import tqdm_notebook as tqdm

In [2]:
GPG_URL = 'https://gender-pay-gap.service.gov.uk/viewing/download-data/2018'

In [3]:
df = pd.read_csv(GPG_URL)
len(df)

10812

In [24]:
df = df[df["CompanyNumber"].notnull()]
len(df)

9474

In [15]:
pc_regex = r'([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2})'

In [20]:
df.loc[:, "Postcode"] = df["Address"].str.extract(pc_regex)[1]

In [33]:
pc = {}
pc_url = 'https://postcodes.findthatcharity.uk/postcodes/{}.json'
for p in tqdm(df["Postcode"].head(20).unique()):
    r = requests.get(pc_url.format(p.replace(" ", "")))
    pc[p] = r.json().get("data", {}).get("attributes", {})

In [39]:
pd.DataFrame(pc).T.columns

Index(['bua11', 'bua11_name', 'buasd11', 'buasd11_name', 'calncv',
       'calncv_name', 'ccg', 'ccg_name', 'ced', 'ced_name', 'ctry',
       'ctry_name', 'cty', 'cty_name', 'dointr', 'doterm', 'eer', 'eer_name',
       'hlthau', 'hlthau_name', 'imd', 'lat', 'laua', 'laua_name', 'lep1',
       'lep1_name', 'lep2', 'lep2_name', 'location', 'long', 'lsoa11',
       'lsoa11_name', 'msoa11', 'msoa11_name', 'nhser', 'nhser_name', 'nuts',
       'nuts_name', 'oa11', 'oac11', 'oac11_name', 'oseast1m', 'osgrdind',
       'osnrth1m', 'park', 'park_name', 'pcd', 'pcd2', 'pcds', 'pcon',
       'pcon_name', 'pct', 'pct_name', 'pfa', 'pfa_name', 'rgn', 'rgn_name',
       'ru11ind', 'ru11ind_name', 'stp', 'stp_name', 'teclec', 'teclec_name',
       'ttwa', 'ttwa_name', 'usertype', 'ward', 'ward_name', 'wz11'],
      dtype='object')

In [42]:
df = df.join(pd.DataFrame(pc).T[["lat", "long", "pcon", "cty", "laua", "ward", "lsoa11", "rgn"]], on='Postcode')

In [23]:
len(df["Postcode"].unique())

7077

In [25]:
df["Postcode"].isnull().value_counts()

False    9445
True       29
Name: Postcode, dtype: int64

In [51]:
print(df[df["rgn"]=="S99999999"].to_json(orient='records'))

[{"EmployerName":"\"RED BAND\" CHEMICAL COMPANY, LIMITED","Address":"19 Smith's Place,\r\nLeith Walk,\r\nEdinburgh,\r\nEH6 8NU","CompanyNumber":"SC016876","SicCodes":"47730","DiffMeanHourlyPercent":7.7,"DiffMedianHourlyPercent":-4.2,"DiffMeanBonusPercent":23.7,"DiffMedianBonusPercent":-75.2,"MaleBonusPercent":22.2,"FemaleBonusPercent":82.0,"MaleLowerQuartile":31.6,"FemaleLowerQuartile":68.4,"MaleLowerMiddleQuartile":12.7,"FemaleLowerMiddleQuartile":87.3,"MaleUpperMiddleQuartile":11.6,"FemaleUpperMiddleQuartile":88.4,"MaleTopQuartile":20.3,"FemaleTopQuartile":79.7,"CompanyLinkToGPGInfo":null,"ResponsiblePerson":"Philip Galt (Managing Director)","EmployerSize":"250 to 499","CurrentName":"\"RED BAND\" CHEMICAL COMPANY, LIMITED","SubmittedAfterTheDeadline":false,"DueDate":"05\/04\/2019 00:00:00","DateSubmitted":"01\/04\/2019 12:37:21","Postcode":"EH6 8NU","lat":55.967066,"long":-3.173121,"pcon":"S14000023","cty":"S99999999","laua":"S12000036","ward":"S13002930","lsoa11":"S01008796","rgn":"